In [1]:
import quakeio
import quakeio.processing as proc
from pathlib import Path
from datetime import datetime
DIR = Path("/mnt/c/Users/claud/Downloads/brace2/")

In [7]:
# config = dict(
#     ch = ("2", "7"),
#     dataset = "meloland",
#     direction="Transverse"
# )
config = dict(
    ch = ("1", "11"),
    # ch = ("3", "7"),
    dataset = "painter",
    direction="Longitudinal"
)
# config = dict(
#     ch = ("23", "25"),
#     # ch = ("3", "7"),
#     dataset = "hayward",
#     direction="Transverse"
# )

In [8]:
def make_hover_data(data, ln=None):
    import numpy as np
    if ln is None:
        items = np.array([d.values for d in data])
        keys = data[0].keys()
    else:
        items = np.array([list(data.values())]*ln)
        keys = data.keys()
    return {
        "hovertemplate": "<br>".join(f"{k}: %{{customdata[{v}]}}" for v,k in enumerate(keys)),
        "customdata": list(items),
    }

In [11]:
res = []
data = []
periods = []
for i in Path(config["dataset"]).glob("*.zip"):
    print(i)
    try:
        collection = quakeio.read(i, exclusions=["filter*"])
    except Exception as e:
        print(f"\tError parsing: {e}")
        continue

    c3 = collection.at(channel=config["ch"][0])
    c7 = collection.at(channel=config["ch"][1])
    try:
        if c3 is None:
            c3 = collection.at(station_channel=config["ch"][0])
        if c3 is None:
            inp = collection.motions['footing_of_center_column']
            try:
                c3 = inp.tran
            except:
                c3 = inp.components['270']

        if c7 is None:
            c7 = collection.at(station_channel=config["ch"][1])
        if c7 is None:
            out = collection.motions['deck_middle_of_bridge']
            try:
                c7 = out.tran
            except:
                c7 = out.components['270']
        print(f"\t{c3.accel['units']}")

        t, s = proc.TransferFunction((c3,c7), per=(0.02, 0.6, 100), damping=[0.0001]).tf()
        T = t[max(s)==s]
        periods.append(T)
        date = c3.get("date",None)
        year = datetime.fromisoformat(date).year if date else "NA"
        res.append({
                "PGA": abs(max(c3.accel.data,key=abs))/980.7,
                "Year": year,
                "Peak": periods[-1],
                "File": i.name
        })
        d = {
            "name": str(year) + f" ({res[-1]['PGA']:5.2}g)",
            "line": {"color":["#003262"]},#, "#808080"][float(T) < 0.1]},
            "type": "scatter",
            "mode": "lines",
            "x": t,
            "y": s,
            **make_hover_data(res[-1], len(t))
        }
        data.append(d)
    except Exception as e:
        print(f"\tError: {e}")


painter/CapeMendocino_21Nov1986_CE89324P.zip
	cm/sec/sec


/tmp/ipykernel_32066/3940025581.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  items = np.array([list(data.values())]*ln)


painter/petrolia_05dec2016_72733405_ce89324p.zip
	cm/sec/sec
painter/RioDell82_CE89324P.zip
	cm/sec/sec
painter/CapeMendocino_31Jul1987_CE89324P.zip
	cm/sec/sec
painter/nc73201181_ce89324p.zip
	cm/sec/sec
painter/RioDell_Petrolia_Processed_Data.zip
	cm/sec/sec
painter/riodell_14sep2012_71842075_ce89324p.zip
	cm/sec/sec
painter/ferndale_28jan2015_72387946_ce89324p.zip
	cm/sec/sec
painter/PetroliaAftershock1_26Apr1992_CE89324P.zip
	cm/sec/sec
painter/Eureka_24Aug1983_CE89324P.zip
	cm/sec/sec
painter/Trinidad-Offshore_08Nov1980_CE89324p.zip
	cm/sec/sec
painter/riodell_19oct2014_72330211_ce89324p.zip
	cm/sec/sec
painter/ferndaleoffshore_08dec2016_us20007z6r_ce89324p.zip
	cm/sec/sec
painter/CapeMendocinoAftershock_21Nov1986_CE89324P.zip
	cm/sec/sec
painter/bayview_11oct2013_72086051_ce89324p.zip
	cm/sec/sec


In [12]:
import pandas as pd
import numpy as np

pd.DataFrame(periods).describe()
with open(DIR/f"{config['dataset']}-table-{'-'.join(config['ch'])}.html", "w+") as f:
    f.write(pd.DataFrame(periods).describe().to_html())

In [13]:
max_accel = abs(max((r["PGA"] for r in res), key=abs))
min_accel = abs(min((r["PGA"] for r in res), key=abs))
for d,r in zip(data, res):
    d["line"] = {
        "color": "rgba(0, 50, 98,{})".format(
            0.1+(r["PGA"]-min_accel)/(max_accel-min_accel)*0.9
        )
    }

In [ ]:
import plotly.io as pio
pio.renderers.default='iframe'
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline

pyo.init_notebook_mode()

fig = go.Figure(dict(
    data = data,
    layout = go.Layout(),
))
fig.update_layout(
    title=f"{config['dataset']} {config['direction']} History".title()+f" (Ch. {' vs. '.join(config['ch'])})",
    xaxis_title="Period (sec.)",
    yaxis_title="Sa(out)/Sa(in)",
    # legend_title="Legend Title",
    # font=dict(
    #     family="Courier New, monospace",
    #     size=18,
    #     color="RebeccaPurple"
    # )
)
fig.show()
    
# with open(DIR/f"{config['dataset']}-plot-{'-'.join(config['ch'])}.html", "w+") as f:
#     f.write(pio.to_html(fig))


In [ ]:
period = np.stack([
    d["x"][max(d["y"])==d["y"]] for d in data
])
pd.DataFrame(period).describe()